<a href="https://colab.research.google.com/github/VtlNmnk/crema/blob/master/01_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Get access to google drive folder.
# This only needs to be done once per notebook.
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/ACE/'  #change dir to your project folder

# Change current directoty to mounted on gdrive
os.chdir(root_path)
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/My Drive/ACE'

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: 


In [3]:
!ls

 augment_results_collection   output_results_carole_king
'Colab Notebooks'	      output_results_queen
 crema			      output_results_TPU
 datasets		      pumpp
 example		      __pycache__
 libs			      pyrubberband
 log_ACE		      requirements.txt
 muda			      resources
 output_example_results       rubberband-cli_1.8.1-4_amd64.deb
 output_results


In [0]:
# Sampling rate for audio analysis
sample_rate = 44100.

# Hop length for audio analysis
hop_length = 4096

# Number of octaves above C1
n_octaves = 6

# Path for augmented data (optional)
# augment_path = "augment_results_collection/augment_results_michael_jackson"
augment_path = "augment_results_collection/augment_results_carole_king"

# Path for directory containing (audio, jams)
input_path = os.path.join(root_path, "datasets/Isophonics/Carole King")
# input_path = os.path.join(root_path, "datasets/Isophonics/Michael Jackson")
# input_path = os.path.join(root_path, "example")
# input_path = ""

# Path to store pump output
output_path = os.path.join(root_path, "output_results_carole_king")
# output_path = os.path.join(root_path, "output_results_michael_jackson")
# output_path = os.path.join(root_path, "output_example_results")

In [0]:
def make_pump(sr, hop_length, n_octaves):
    p_feature = pumpp.feature.HCQTMag(name='cqt',
                                      sr=sr, hop_length=hop_length,
                                      harmonics=[1, 2],
                                      log=True, conv='tf', n_octaves=n_octaves)

    p_chord_tag = pumpp.task.ChordTagTransformer(name='chord_tag',
                                                 sr=sr, hop_length=hop_length,
                                                 sparse=True)

    p_chord_struct = pumpp.task.ChordTransformer(name='chord_struct',
                                                 sr=sr, hop_length=hop_length,
                                                 sparse=True)

    pump = pumpp.Pump(p_feature, p_chord_tag, p_chord_struct)

    # Save the pump

    with open(os.path.join(output_path, 'pump.pkl'), 'wb') as fd:
        pickle.dump(pump, fd)

    return pump


def convert(aud, jam, pump, outdir, index, list_of_tracks):

    fname = os.path.extsep.join([os.path.join(outdir, crema_utils.base(aud)),
                                'h5'])
    # this is in case some of the files have been converted and some not
    if not os.path.isfile(fname):
        data = pump.transform(aud, jam)
        crema_utils.save_h5(fname, **data)
    list_of_tracks.update({index: crema_utils.base(aud)})
    return list_of_tracks

In [0]:
# !dpkg -i rubberband-cli_1.8.1-4_amd64.deb
# !pip install jams
# !pip install pumpp
# !pip install soundfile
# !pip install jsonpickle
# !pip install crema
# !pip uninstall muda

In [7]:
# import crema
# from crema.utils import get_ann_audio
import json
from tqdm import tqdm
import pickle
from joblib import Parallel, delayed

try:
    import librosa
except ImportError:
    !pip install -q librosa==0.6.2
    import librosa
try:
    from jams.util import smkdirs
except ImportError:
    !pip install -q jams
    from jams.util import smkdirs

try:
    import pumpp
except ImportError:
    !pip install -q pumpp
    import pumpp

from crema import utils as crema_utils

smkdirs(output_path)

pump = make_pump(sample_rate, hop_length, n_octaves)

index = 0
list_of_tracks = {}

if input_path:
    loop = tqdm(crema_utils.get_ann_audio(input_path),
                desc='Converting training data')
    for aud, ann  in loop:
        convert(aud, ann, pump, output_path, index, list_of_tracks)
        index +=1

if augment_path:
    loop = tqdm(crema_utils.get_ann_audio(augment_path),
             desc='Converting augmented data')
    for aud, ann  in loop:
        convert(aud, ann, pump, output_path, index, list_of_tracks)
        index +=1
file_path = os.path.join(output_path, "index_train.json")
with open(file_path, 'w') as outfile:
    print("\n")
    print("Writing file to: ", file_path)
    json.dump({"id":list_of_tracks}, outfile)

Converting augmented data: 100%|██████████| 132/132 [43:18<00:00, 19.79s/it]



Writing file to:  /content/gdrive/My Drive/ACE/output_results_carole_king/index_train.json
